In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
from google.colab import files
import io
!pip install -U tensorflow-addons
import tensorflow_addons as tfa


input_size = 64
output_size = 10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.1 MB/s 


In [2]:
#Set the values for the number of sublayers and loss
sublayers = 64 #To ensure full expressivity
gamma_h = 0.0 #Lossless - corresponds to 0 GHz

In [6]:
#Cell to import and pre-process the data

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
len_training_data = 50000
len_testing_data = 5000

x_train = x_train[:len_training_data]
y_train = y_train[:len_training_data]
x_test = x_test[len_testing_data:]
y_test = y_test[len_testing_data:]

num_batches = 25
train_batch_size = int(len_training_data/num_batches)
test_batch_size = int(len_testing_data/num_batches)

n = int(np.sqrt(input_size)/2)
img_data_re = np.pad(np.real(np.fft.fftshift(np.fft.fft2(x_train[0])))[14-n:14+n, 14-n:14+n], [(14-n,14-n), [14-n,14-n]], mode = 'constant') 
img_data_im = np.pad(np.real(np.fft.fftshift(np.fft.fft2(x_train[0])))[14-n:14+n, 14-n:14+n], [(14-n,14-n), [14-n,14-n]], mode = 'constant') 
img_data = img_data_re + 1j*img_data_im
img = np.abs(np.fft.ifft2(img_data))

#Taking Fourier Transform - NN classifying central window of Fourier Transform on MNIST images
#Splitting and Structuring the training and testing data

def img_data_norm(index, data):
  return np.abs(np.fft.fftshift(np.fft.fft2(data[index])))[14-n:14+n, 14-n:14+n]
def img_data_phase(index, data):
  return np.angle(np.fft.fftshift(np.fft.fft2(data[index])))[14-n:14+n, 14-n:14+n]

x_train_data = np.zeros([len_training_data, input_size], dtype = "complex64")
for i in range(len_training_data):
  x_train_data[i] = x_train_data[i] + np.reshape(img_data_norm(i, x_train)*np.exp(1j*img_data_phase(i, x_train)), (1, input_size))

y_train_data = np.zeros([len_training_data, 10], dtype = np.float32)
num = np.ones([1], dtype = "float32")
for i in range(len_training_data):
  temp = np.zeros([10], dtype = np.float32)
  temp[y_train[i]] = temp[y_train[i]] + num
  y_train_data[i] = temp

x_test_data = np.zeros([len_testing_data, input_size], dtype = "complex64")
for i in range(len_testing_data):
  x_test_data[i] = x_test_data[i] + np.reshape(img_data_norm(i, x_test)*np.exp(1j*img_data_phase(i, x_test)), (1, input_size))

y_test_data = np.zeros([len_testing_data, 10], dtype = np.float32)
num = np.ones([1], dtype = "float32")
for i in range(len_testing_data):
  temp = np.zeros([10], dtype = np.float32)
  temp[y_test[i]] = temp[y_test[i]] + num
  y_test_data[i] = temp

##NOTE: Scaling input data to have max of 20 since the nonlinearity is capped at 20.2

y_train_data = (tf.convert_to_tensor(y_train_data))
y_train_tensors = tf.convert_to_tensor(tf.split(y_train_data, num_or_size_splits = num_batches, axis = 0))
x_train_data = tf.convert_to_tensor(tf.transpose(x_train_data), dtype = tf.complex64)
x_train_data = (x_train_data)/(np.max(np.abs(x_train_data))) * 50
x_train_tensors = tf.convert_to_tensor(tf.split(x_train_data, num_or_size_splits = num_batches, axis = 1))

y_test_data = (tf.convert_to_tensor(y_test_data))
y_test_tensors = tf.convert_to_tensor(tf.split(y_test_data, num_or_size_splits = num_batches, axis = 0))
x_test_data = tf.convert_to_tensor(tf.transpose(x_test_data), dtype = tf.complex64)
x_test_data = (x_test_data)/(np.max(np.abs(x_test_data))) * 50
x_test_tensors = tf.convert_to_tensor(tf.split(x_test_data, num_or_size_splits = num_batches, axis = 1))

11490434/11490434 [==============================] - 0s 0us/step


In [7]:
#Cell to import the Nonlinear activation function values generated from Julia file
#Using nonlins_0.npz as the sample nonlinearity for now
uploaded = files.upload()
data_path = 'nonlins_0.npz'
nonlinear = np.load(data_path)

##Selecting kze_s = 0.2 as nonlinearity for now (index 0/1/2 corresponds to 0.0/0.1/0.2)
nl_data = np.transpose(nonlinear['allA'], [2, 0, 1])[2]

Saving nonlins_0.npz to nonlins_0.npz


In [8]:
#Cell to unroll the nonlinearity and prepare for use in interpolation
nl_amps = tf.math.abs(nl_data)
nl_phase = tf.math.angle(nl_data)
phis = nonlinear['phis']
en_to_es = nonlinear['en_over_es']

delta_phi = (phis[1] - phis[0])
delta_rat = (en_to_es[1] - en_to_es[0])
             
#Input grid to the interpolation function
ref_amps_tensor = np.reshape(nl_amps, [1, *nl_amps.shape, 1])

nl_phase_unwrap = np.unwrap(nl_phase, axis = 0)
nl_unwrap_concat = tf.concat([nl_phase_unwrap + 2* np.pi, nl_phase_unwrap[1:]], axis = 0)
ref_phase_tensor = np.reshape(nl_unwrap_concat, [1, *nl_unwrap_concat.shape, 1])

In [9]:
##Creating the basis Toeplitz matrices of sizes 64, 10 - modify based on network architecture

N = 64
P = N-1
toeplitz_basis_matrices_64 = np.array([np.diag(np.ones(N - 1 - i), k = i + 1) for i in range(0, N - 1)])
toeplitz_basis_matrices_64 = tf.constant(toeplitz_basis_matrices_64, dtype = tf.complex64)

N = 10
P = N-1
toeplitz_basis_matrices_10 = np.array([np.diag(np.ones(N - 1 - i), k = i + 1) for i in range(0, N - 1)])
toeplitz_basis_matrices_10 = tf.constant(toeplitz_basis_matrices_10, dtype = tf.complex64)

In [10]:
#Cell to modify annealing rate of Gamma - presently is set to constant
anneal_degree = 0
dummy_range = tf.range(0, 10+10/5000, 10/4999)
gamma_scaled = (gamma_h)/(10**anneal_degree)
Gamma = tf.cast(gamma_scaled*(dummy_range)**anneal_degree, dtype = tf.complex64)

In [15]:
#Cell to make the neural network

class Network(keras.layers.Layer):
  def __init__(self):
    super(Network, self).__init__()
    #Architecture: (N sublayers * input number of freq states) -> (1 sublayer * 10 output freq states)
    self.pumps_1 = tf.Variable(tf.complex(tf.random.normal(shape = [sublayers, input_size - 1]), 0.0), trainable = True)
    self.pump_diag_1 = tf.Variable((tf.random.normal(shape = [sublayers])), trainable = True)
    self.pumps_2 = tf.Variable(tf.complex(tf.random.normal(shape = [1, output_size -1]), 0.0), trainable = True)
    self.pump_diag_2 = tf.Variable((tf.random.normal(shape = [1])), trainable = True)

  def make_toeplitz(self, toeplitz_basis_matrices, pumps, pump_diag, depth, size, epoch_num, gamma_1):
    #Make linear transforms - exponents of Toeplitz matrices
    toeplitz_basis_matrices = tf.repeat([toeplitz_basis_matrices,], repeats = depth, axis = 0)
    pump_matrix = tf.einsum("ij, ijkl -> ikl", pumps, toeplitz_basis_matrices)
    pump_matrix = pump_matrix - tf.transpose(tf.math.conj(pump_matrix), [0,2,1])
    pump_matrix = tf.repeat([tf.complex(tf.eye(size), 0.0),], repeats = depth, axis = 0)*(-gamma_1[epoch_num]/2) + pump_matrix    ##This gives the toeplitz matrix
    pump_matrix = pump_matrix + tf.einsum('i, ijk -> ijk', 1j*tf.cast(pump_diag**2, dtype = tf.complex64), tf.repeat([tf.eye(size, dtype = tf.complex64)], repeats = depth, axis = 0))
    #print (pump_matrix + tf.math.conj(tf.transpose(pump_matrix, [0, 2, 1])))
    pump_matrix = tf.linalg.expm(pump_matrix)
    U = tf.eye(size, dtype = tf.complex64)
    for i in range(depth):
      U = tf.matmul(pump_matrix[i], U)
    return U

  def nonlinearity(self, linear_input):
    #Nonlinear activation function
    #Extract phase and amplitude data from the linear input after a FWM
    phases = tf.math.angle(linear_input)
    amps = tf.math.abs(linear_input)
    #Reshaping for the interpolation
    phase_in = tf.reshape(phases/delta_phi, shape = [1, tf.size(linear_input)]) + 36 ##Because we have concat, we do offset
    amps_in = tf.reshape(amps/delta_rat, shape = [1, tf.size(linear_input)])
    #Structure the query data for interpolation
    amps_phase_in = [tf.transpose(tf.concat([phase_in, amps_in], axis = 0))]
    #Doing the interpolation
    amps_out = tfa.image.interpolate_bilinear(ref_amps_tensor, query_points = amps_phase_in, indexing = 'ij') 
    phase_out = tfa.image.interpolate_bilinear(ref_phase_tensor, query_points = amps_phase_in, indexing = 'ij')
    #Putting the data back together and restructuring
    neuron_out = tf.cast(amps_out, dtype = tf.complex64) * tf.math.exp(1j * tf.cast(phase_out, dtype = tf.complex64))
    neuron_out = tf.reshape(neuron_out, shape = tf.shape(linear_input))
    return neuron_out

  def call(self, data, len_training_data, epoch_num, gamma_1):
    #Run the neural network
    #First layer with 64 neurons
    layer_1_out = tf.matmul(self.make_toeplitz(toeplitz_basis_matrices_64, self.pumps_1, self.pump_diag_1, sublayers, 64, epoch_num, gamma_1), data) 
    '''Typicially energy conversation constant should be added here, but we can approximate it to 1'''
    layer_1_out = self.nonlinearity(tf.slice(layer_1_out, [0,0], [10, len_training_data]))
    #layer_1_out = tf.slice(layer_1_out, [0,0], [10, len_training_data])
    #Second layer with 10 neurons
    layer_2_out = tf.matmul(self.make_toeplitz(toeplitz_basis_matrices_10, self.pumps_2, self.pump_diag_2, 1, 10, epoch_num, gamma_1), layer_1_out)
    '''Typicially energy conversation constant should be added here, but we can approximate it to 1'''
    #NOTE: No nonlinearity after the final layer
    return tf.nn.softmax(tf.abs(tf.transpose(layer_2_out)))
    
epoch_num = 0
NN_function = Network()
a = NN_function(x_train_tensors[0], train_batch_size, 24, Gamma)

In [16]:
NN_function = Network()
#200 epochs on the cluster
num_epochs = 200
train_loss = np.zeros([num_epochs * 25], dtype = np.float16)
train_acc = np.zeros([num_epochs * 25], dtype = np.float16)
train_acc_batch = np.zeros([num_epochs], dtype = np.float16)
test_acc_batch = np.zeros([num_epochs], dtype = np.float16)
train_loss_batch = np.zeros([num_epochs], dtype = np.float16)
test_loss_batch = np.zeros([num_epochs], dtype = np.float16)
##Learning rate and decay rate here
lr = 0.005
decay = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, 5000, decay)
optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
step_no = 0

'''Gradient Descent starts here'''
for epoch in range(num_epochs):
  print ("Epoch number: " + str(epoch))
  for step in range(num_batches):
    with tf.GradientTape() as tape:
      y_pred_train = NN_function(x_train_tensors[step], train_batch_size, epoch_num = step_no, gamma_1 = Gamma)
      #Loss function calculation here
      loss_value_train = tf.math.reduce_mean(tf.keras.losses.MSE(y_pred_train, y_train_tensors[step])) #+ lambda_l2*(tf.math.reduce_mean(w0**2) + tf.math.reduce_mean(w1**2))# + lambda_l1*(tf.math.reduce_mean(w0) + tf.math.reduce_mean(w1)) 
      gradients = tape.gradient(loss_value_train, NN_function.trainable_variables)
    #Accuracy calculation here
    acc_value_train = tf.math.reduce_mean(tf.keras.metrics.categorical_accuracy(y_train_tensors[step], y_pred_train))
    print ("Step : " + str(step_no) + ", Loss : " + str(loss_value_train.numpy()) + ", Accuracy : " + str(acc_value_train.numpy()))
    optimizer.apply_gradients(zip(gradients, NN_function.trainable_variables))
    step_no = optimizer.iterations.numpy()
    train_loss[step_no - 1] = train_loss[step_no - 1] + loss_value_train
    train_acc[step_no - 1] = train_acc[step_no - 1] + acc_value_train
    print ('')
    

Streaming output truncated to the last 5000 lines.
Step : 2549, Loss : 0.013006174, Accuracy : 0.9195

Epoch number: 102
Step : 2550, Loss : 0.011295067, Accuracy : 0.933

Step : 2551, Loss : 0.009070222, Accuracy : 0.948

Step : 2552, Loss : 0.010226224, Accuracy : 0.9455

Step : 2553, Loss : 0.010771301, Accuracy : 0.9415

Step : 2554, Loss : 0.011471891, Accuracy : 0.9305

Step : 2555, Loss : 0.0099833105, Accuracy : 0.946

Step : 2556, Loss : 0.012555693, Accuracy : 0.926

Step : 2557, Loss : 0.01305484, Accuracy : 0.9185

Step : 2558, Loss : 0.011340764, Accuracy : 0.9375

Step : 2559, Loss : 0.009182053, Accuracy : 0.9495

Step : 2560, Loss : 0.010445617, Accuracy : 0.943

Step : 2561, Loss : 0.010945051, Accuracy : 0.935

Step : 2562, Loss : 0.0113829225, Accuracy : 0.933

Step : 2563, Loss : 0.011543694, Accuracy : 0.9335

Step : 2564, Loss : 0.011914352, Accuracy : 0.9295

Step : 2565, Loss : 0.01336934, Accuracy : 0.9165

Step : 2566, Loss : 0.010970219, Accuracy : 0.9385

St